基础API使用

In [1]:
# 使用tf.keras搭建回归模型，数据集使用加利福尼亚的房价预测
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
# 从内存中构建数据集
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))
print(dataset)

<TensorSliceDataset shapes: (), types: tf.int32>


In [3]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [4]:
#1、重复读取数据集/repeat epoch
#2、get batch,数据集分块
dataset = dataset.repeat(3).batch(7) #遍历三次，每次返回7个数据
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [7]:
# interleave:对dataset中每个元素处理后，形成一个新的数据集
# case:遍历文件中的文件名，读取文件内容，
#每个文件名形成一个新的数据集，interleave把新的数据集合并起来，成文一个总数据集

dataset2 = dataset.interleave(
    # 每次从dataset中取出来一个v,并把v封装成一个新的数据集
    lambda v: tf.data.Dataset.from_tensor_slices(v),# map_fn,做什么样的变化
    cycle_length = 5,#cycle_length，并行程度
    block_length = 5# block_length 每次取多少个元素出来
)
for item in dataset2:
    print(item)
# 均匀混合的效果

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype

In [8]:
# from_tensor_slices也支持元组和字典
x = np.array([[1,2],[3,4],[5,6]])# 向量
y = np.array(['cat','dog','fox'])# 类别
# 两个元素的第一个维度一定要相等，长度都为3
dataset3 = tf.data.Dataset.from_tensor_slices((x,y))# 输入为元组
print(dataset3)

for item_x, item_y in dataset3:
    print(item_x.numpy(), item_y.numpy())

<TensorSliceDataset shapes: ((2,), ()), types: (tf.int32, tf.string)>
[1 2] b'cat'
[3 4] b'dog'
[5 6] b'fox'


In [11]:
dataset4 = tf.data.Dataset.from_tensor_slices({
    'feature':x,
    'label':y
})
for item in dataset4:
    print(item)
    print(item['feature'].numpy(),item['label'].numpy())

{'feature': <tf.Tensor: id=316, shape=(2,), dtype=int32, numpy=array([1, 2])>, 'label': <tf.Tensor: id=317, shape=(), dtype=string, numpy=b'cat'>}
[1 2] b'cat'
{'feature': <tf.Tensor: id=322, shape=(2,), dtype=int32, numpy=array([3, 4])>, 'label': <tf.Tensor: id=323, shape=(), dtype=string, numpy=b'dog'>}
[3 4] b'dog'
{'feature': <tf.Tensor: id=328, shape=(2,), dtype=int32, numpy=array([5, 6])>, 'label': <tf.Tensor: id=329, shape=(), dtype=string, numpy=b'fox'>}
[5 6] b'fox'
